## <span style="color:red">Homework 2 </span>

**I- Reading Assignment**

[James, G., Witten, D., Hastie, T., Tibshirani, R., & Taylor, J. (2023). An Introduction to Statistical Learning : With Applications in Python (1st ed. 2023 edition). Springer.](https://www.statlearning.com/)

- Read Chapter 8, Section 1 (8.1: pages 331-342)


In [38]:
import os 
import pandas as pd

import jupyter_black
jupyter_black.load()

# ignore warning
import warnings
warnings.filterwarnings("ignore")

**II- Binary Classification**

1- Select your binary outcome

Amongst your categorical variables, select a binary variable to be used as an outcome.
It you have no binary variable, then create a binary variable from one of your categorical variables.

2- Divide your dataset into a test sample and a training sample with a ratio of 20% vs 80% respectively. <br>
Be careful to do so that your outcomes remain representative of the initial data set and well shuffled.

3- Select one classifier and fit it on the training set then estimate its score both on the training set and the test set. <br>Use the accuracy, the ROC AUC, the F1 score and the Kappa to estimate the performance of your classifier.

4- Repeat the same code on two other different classifiers: 

- Recommendation: create a function that does all the job and run it using three different classifiers
. 
- Which is the best classifier ?

In [3]:
os.getcwd()

'/Users/fulin/Documents/GitHub/EMLyon_DSAIS/ML/homework02'

In [4]:
PATH = '/Users/fulin/Documents/GitHub/EMLyon_DSAIS/ML/homework01/data'
os.chdir(PATH)

In [18]:
train_df = pd.read_csv('customer_data_train.csv', index_col=0)
test_df = pd.read_csv('customer_data_test.csv', index_col=0)

In [25]:
test_df

,account_length,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,state,area_code,international_plan,voice_mail_plan,churn
0,101.0,0.0,70.9,123.0,12.050,270.26,73.0,22.976,236.0,73.0,10.62,10.6,3.0,2.860,3.0,HI,510.0,no,no,no
1,137.0,0.0,223.6,86.0,37.912,244.80,139.0,20.810,94.2,81.0,4.24,9.5,7.0,2.570,0.0,MT,510.0,no,no,no
2,103.0,29.0,294.7,95.0,40.878,237.30,105.0,20.170,300.3,127.0,13.51,13.7,4.8,3.700,1.0,OH,408.0,no,yes,no
3,99.0,33.2,216.8,123.0,36.860,126.40,88.0,10.740,220.6,82.0,9.93,9.0,2.0,2.432,1.0,NM,415.0,no,no,no
4,108.0,0.0,197.4,78.0,33.560,124.00,101.0,10.540,204.5,107.0,9.20,7.7,4.0,2.080,2.0,SC,415.0,no,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3861,136.0,29.0,85.2,98.0,14.480,173.82,85.0,19.580,243.6,104.0,10.96,9.0,3.0,2.430,2.0,DE,408.0,yes,yes,no
3862,68.0,34.0,160.0,108.2,27.200,184.50,119.0,18.214,208.3,101.0,9.37,6.1,6.2,1.650,1.0,MS,415.0,no,yes,no
3863,132.0,10.0,182.9,54.0,31.090,292.40,94.8,24.850,142.3,116.0,6.40,11.5,4.0,3.110,0.0,OH,415.0,no,yes,no
3864,133.0,0.0,216.2,79.8,36.750,222.20,133.0,18.890,192.0,95.0,8.64,3.1,1.0,0.840,2.0,LA,415.0,no,no,no


In [28]:
cbn_df = train_df.append(test_df)

In [29]:
cbn_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12873 entries, 3866 to 3865
Data columns (total 20 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   account_length                 12873 non-null  float64
 1   number_vmail_messages          12873 non-null  float64
 2   total_day_minutes              12873 non-null  float64
 3   total_day_calls                12873 non-null  float64
 4   total_day_charge               12873 non-null  float64
 5   total_eve_minutes              12873 non-null  float64
 6   total_eve_calls                12873 non-null  float64
 7   total_eve_charge               12873 non-null  float64
 8   total_night_minutes            12873 non-null  float64
 9   total_night_calls              12873 non-null  float64
 10  total_night_charge             12873 non-null  float64
 11  total_intl_minutes             12873 non-null  float64
 12  total_intl_calls               12873 non-nul

In [35]:
predictors = cbn_df.drop(columns=['churn'], axis=1)
outcome_ready = cbn_df['churn'].map({"yes":1,"no":0})

In [33]:
quanti_features = predictors.iloc[:,:14].columns.tolist()
cat_features = predictors.iloc[:,15:].columns.tolist()
cat_ohe_features = predictors.iloc[:,16:].columns.tolist()
cat_svd_features = ['state']

In [34]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import TruncatedSVD

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), quanti_features),
        ('cat_ohe', OneHotEncoder(), cat_ohe_features),
        ('cat_svd', Pipeline([
            ('encoder', OneHotEncoder()),
            ('svd', TruncatedSVD(n_components=4))
        ]), cat_svd_features)
    ],
    remainder='drop'
)

pipeline = Pipeline([
    ('preprocessor', preprocessor)
])

In [37]:
predictors_ready = pipeline.fit_transform(predictors)
display(predictors_ready.shape)

(12873, 25)

In [40]:
from sklearn.model_selection import train_test_split

X_train_1, X_test_1, y_train_1, y_test_1 = train_test_split(
    predictors_ready, outcome_ready, test_size=0.2, shuffle=True, random_state=42
)

X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(
    predictors_ready, outcome_ready, test_size=0.8, shuffle=True, random_state=42
)

**III- Multiclass Classification** (Optional with good bonus point)

This time, select a categorical variable having more than 2 categories. I you have none, then categorize one of your quantitative data (e.g in using `pd.cut()` and the corresponding quantiles) into at least three classes.

Copy the codes of PART I and adjust to the new outcome. Use only Accuracy and F1 Score for performance (or sklearn `classification_report`)